# Effect of Vanishing Policies in traffic rerouting

**Objective**
* The objective is to perform an analysis to determine the impact of introducing vanishing policies in specific zones of a city. 

<img src="../images/zones.png" alt="drawing" width="400"/>


In [ ]:
# Parameters control

# Experiment name 
EXPERIMENT = "CTR"
# Control algorithm 
CTR_ALG = "CO_P"
# Control mode (manual/auto)
CONTROL_MODE = 'AUTO'
# Control distance action
DISTANCE_CONTROL = 800
# Trigger time of an action (Only manual)
TRIGGER_TIME = 9000
# Weight coefficient cooperative control
SELFISH = 0.5
# Network number of rows / columns (Manhattan case)
NRow = 5
NCol = 5
# Control Cooperative proportional
CO_KP = 0.1
# Control Cooperative integral
CO_TI = 360.0
# Control Cooperative Windup
CO_TWD = 1 
# Ego Proportional
KP = 0.1
# Ego Integral 
TI = 360.0
# Ego Windup
TWD = 1

# Scenario values 

# Default path for simulator
PATH_SYMUVIA = '/Users/andresladino/Documents/01-Code/04-Platforms/dev-symuvia/build/lib/libSymuVia.dylib' 
# Scenario 
SCENARIO = 'mesh30x30/'
# Number of zones
ZONES = '25zones/'
# Simulation file 
FILE = 'symuvia_network_25zones.xml'
# Demand file
DEMAND_FILE = 'extnewtripset.csv'
# Reference speeds file
REF_SPEED = "ref_speeds_25zones.csv"

In [ ]:
import symupy
import pandas as pd
import numpy as np 
import os
import sys
import re
from itertools import repeat, chain
import ipywidgets as widgets
from IPython.display import display
import networkx as nx

try:
    pd.set_option('plotting.backend', 'hvplot')
except: 
    pass
print(f"Backend: {pd.options.plotting.backend}")

from symupy.api import Simulator,Simulation

print(f"Version of symupy: {symupy.__version__}")

packages = ['src']

# Adding supplementary functions 
for pck in packages:
    print(f"Adding folder: {pck}")
    sys.path.append(os.path.join(os.getcwd(),f"../{pck}"))


In [ ]:
# Path information

PATH_SCENARIO = os.getcwd()+f"/../data/scenarios/{SCENARIO}{ZONES}{FILE}" # Github
print(f"Scenario: {PATH_SCENARIO}")
DATA_INPUT = f"../data/scenarios/{SCENARIO}" # Github
print(f"Input Data: {DATA_INPUT}")
PATH_SPEEDS = os.getcwd()+f"/../data/scenarios/{SCENARIO}{ZONES}{REF_SPEED}" # Github
print(f"Ref Speeds: {PATH_SPEEDS}")

#### Defining output path & creation

In [ ]:
def create_output_dir():
    """
        Creates output directory for the results 
    """
    output_dir = os.getcwd()+f"/../data/results/{SCENARIO}{ZONES}{EXPERIMENT}_{CTR_ALG}/"
    return output_dir
    

In [ ]:
output_dir = create_output_dir()
print(f"Output Dir: {output_dir}")

In [ ]:
!mkdir -p $output_dir

#### Copy XML file and updating path

In [ ]:
!cp $PATH_SCENARIO $output_dir

In [ ]:
PATH_SCENARIO=f"{output_dir}{FILE}"
print(f"New Path: {PATH_SCENARIO}")

#### File reading

* Read zone links and construct a list of strings by containing links per zone `zone2_Cpt_*.csv`
* Read demand file `extnewtripset.csv`

In [ ]:
# Demand information
demand = pd.read_csv(DATA_INPUT+DEMAND_FILE,sep=";") 
demand.rename({'Unnamed: 0':'vehid'}, axis=1, inplace=True) # Rename column
# demand.head()

In [ ]:
# Some idea of the flow
TIME_MAX = demand.creation.max()
TIME_MIN = demand.creation.min()
DELTA_TIME = TIME_MAX - TIME_MIN
interval_cut = pd.cut(demand.creation,int(DELTA_TIME/60))
flow = demand.groupby(interval_cut).count()
flow['stamp'] = np.arange(len(flow))
# flow.plot(x='stamp',y='creation',grid = True, title = 'Aggregated Demand')

In [ ]:
# Vehicle creation
numveh = demand.groupby(interval_cut).max()
numveh['stamp'] = np.arange(len(numveh))
# numveh.plot(x='stamp',y='creation',grid = True, title = 'Aggregated Demand')

## Zone information 

Taking zone information from CSV files

In [ ]:
# # Zone information

# files = [f for f in os.listdir(DATA_INPUT) if re.search(r'\d.csv$', f)]

# # Data management
# df_zones = [] 

# # Communicate to Symuvia
# dct_ctrl = {}

# # Reading all files
# for file in files: 
#     key = file.split("_")[-1][0] # Zone number
#     df_zone = pd.read_csv(DATA_INPUT+file, header = None, names = ["Link"])
#     df_zone["Zone"] = key
#     df_zones.append(df_zone)
#     ls_links = ' '.join(df_zone["Link"].to_list())
#     dct_ctrl[key] = f"'{ls_links}'" # This special to have "'x'" instead of 'x'

# # All zone data is here
# df_zones = pd.concat(df_zones)
# # df_zones.head()

#### Recreation of a demand

As it can be observed the demand profile here assumed is only valid for the first 150 minutes. In order to reach equilibria we propose to sample vehicles uniformly within this interval to create the amount of vehicles for the following 100 seconds. 

In [ ]:
# TIME_LIMIT = 150
# constant_demand = demand.query(f"creation<={TIME_LIMIT * 60}")
# id_new_veh = constant_demand.vehid.max()

# # Creating new demand 
# extra_demand = constant_demand.copy()
# # Shifting new vehicles instant
# extra_demand['creation'] = extra_demand['creation'].apply(lambda x: x + TIME_LIMIT * 60)
# extra_demand['vehid'] = extra_demand['vehid'].apply(lambda x: x + id_new_veh)
# # Shifting vehicle instant creation 
# extra_demand.head()

# TIME_MAX = extra_demand.creation.max()
# TIME_MIN = extra_demand.creation.min()
# DELTA_TIME = TIME_MAX - TIME_MIN

# interal_cut = pd.cut(extra_demand.creation, bins = int(DELTA_TIME/60))

# extra_flow = extra_demand.groupby(interal_cut).count()
# extra_flow['stamp'] = np.arange(len(extra_flow))
# # extra_flow.plot(x='stamp',y='creation',grid = True, title = 'Extended Demand')

In [ ]:
# demand = pd.concat([constant_demand, extra_demand])

# MAX_TIME = demand.creation.max()

# flow = demand.groupby(pd.cut(demand.creation,int(demand.creation.max()/60))).count()
# flow['stamp'] = np.arange(len(flow))
# # flow.plot(x='stamp',y='creation',grid = True, title = 'Aggregated Demand')



### Launch simulation 

The objective of the following part is to perform a step by step simulation in multiple scenarios. 

#### Scenario generation 

This instantiate and generate the scenario for the simulation 

In [ ]:
# Simulation creation 
simulator = Simulator(PATH_SYMUVIA, writeXML = True)

# Register simulation
simulator.register_simulation(PATH_SCENARIO)

# Sensors
sensors = list(simulator.simulation.get_mfd_sensor_names())
sensor_global = sensors[-1]
# sensors.pop(); # takes out Global

#### Supplementary functions

Define all supplementary functions required for the control 

In [ ]:
from data.transformer import extract_veh_data
from algorithms.control import define_grid_graph, ComputeVanishingControl

# Progress bar 
max_count = 100

# progress =  widgets.IntProgress(
#     value=0,
#     min=0,
#     max=max_count,
#     step=1,
#     description='Progress:',
#     bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
#     orientation='horizontal'
# ) # instantiate the bar  

refSpeed = pd.read_csv(PATH_SPEEDS, header=None)

DEFAULT_SPD = dict(zip(sensors, refSpeed.to_numpy().T[0]))

# Create cooperative graph
G = define_grid_graph(NRow, NCol, selfishness=SELFISH, refSpeed=DEFAULT_SPD, sensors=sensors)

# nx.draw(G, node_color="#A0CBE2", with_labels=True) # Plot


#### Control policy

Here we set the control policy 

In [ ]:
# Control

# Control time
control_interval = 180 # seconds

# Manual defaults
state_A = dict(zip(sensors, repeat(0)))

state_B = dict(zip(sensors, repeat(0)))

# Setting parameters for control action
if CTR_ALG == "CO_P":
    kP = CO_KP
    Ti = CO_KP
    Twd = CO_KP
elif CTR_ALG == "CO_PI":
    kP = CO_KP 
    Ti = CO_TI
    Twd = CO_TWD
else: 
    kP = KP
    Ti = TI
    Twd = TWD

    
if CONTROL_MODE == "MANUAL":
    compute_control = lambda time, threshold, speed, graph: state_B if time>threshold else state_A 
elif CONTROL_MODE == "AUTO": 
    compute_control = ComputeVanishingControl(G, control_interval, kP, Ti, Twd, CTR_ALG)
else:
    compute_control = lambda time, threshold, speed, graph: state_A

    
# Distance control action
dstcontrol = dict(zip(sensors, repeat(DISTANCE_CONTROL)))

#### Runtime simulations

The following launches the runtime simulations.

In [ ]:
# Runtime
TTT = []
TTD = []
SPD = []
CTR = []

TTT_GLOBAL = []
TTD_GLOBAL = []
SPD_GLOBAL = []

# Simulation runtime
# display(progress) # display the bar

vehids = []
cum_veh = []

with simulator as s:
#     progress.value = 0
    while s.do_next:

        # Vehicle Creation on demand
        if s.simulationstep > 0:
            for veh_data in extract_veh_data(demand, s.simulationstep):
                vehid = s.create_vehicle_with_route(*veh_data)
                vehids.append(vehid)

        # Get measurement
        if not s.simulationstep % control_interval and s.simulationstep > 0:
            
            TTD.append(dict(zip(sensors[:-1], s.get_total_travel_distance(sensors[:-1]))))
            TTT.append(dict(zip(sensors[:-1], s.get_total_travel_time(sensors[:-1]))))
            SPD.append(dict(zip(sensors[:-1], s.get_mfd_speed(sensors[:-1]))))

            TTT_GLOBAL.append({sensor_global: s.get_total_travel_distance(sensor_global)})
            TTD_GLOBAL.append({sensor_global: s.get_total_travel_time(sensor_global)})
            SPD_GLOBAL.append({sensor_global: s.get_mfd_speed(sensor_global)})                
                
        # Declare Zone Control
        if s.simulationstep == control_interval:
            control_rate = compute_control(s.simulationstep, TRIGGER_TIME, SPD, G)
            CTR.append(control_rate)
            s.add_control_probability_zone_mfd(control_rate, dstcontrol)

        # Modify Zone Control
        if not s.simulationstep % control_interval and s.simulationstep > control_interval:
#             progress.value += 1
            # Control
            new_vehs = [v for v in vehids if v not in cum_veh and v >= 0]
            fail_vehs = [v for v in vehids if v not in cum_veh and v < 0]
            print(
                f"Time: {s.simulationstep},New vehs: {len(new_vehs)}, Failed vehs:{len(fail_vehs)}, Total vehs:{len(vehids)}"
            )

            # Store new vehicles
            cum_veh = [v for v in vehids]        

            control_rate = compute_control(s.simulationstep, TRIGGER_TIME, SPD, G)
            CTR.append(control_rate)

            s.modify_control_probability_zone_mfd(control_rate)

        s.run_step() 

In [ ]:
# Results saving

# DataFrames
ttd = pd.DataFrame(TTD)
ttt = pd.DataFrame(TTT)
spd = pd.DataFrame(SPD)
ctr = pd.DataFrame(CTR)

# Global 
ttdG = pd.DataFrame(TTT_GLOBAL)
tttG = pd.DataFrame(TTD_GLOBAL)
spdG = pd.DataFrame(SPD_GLOBAL)

ttt.to_csv(output_dir + 'ttt.csv',index=False)
ttd.to_csv(output_dir + 'ttd.csv',index=False)
spd.to_csv(output_dir + 'spd.csv',index=False)
ctr.to_csv(output_dir + 'ctr.csv',index=False)

tttG.to_csv(output_dir + 'tttG.csv',index=False)
ttdG.to_csv(output_dir + 'ttdG.csv',index=False)
spdG.to_csv(output_dir + 'spdG.csv',index=False)

In [ ]:
# Remove original file 
!rm -f $PATH_SCENARIO

Andres L.